In [1]:
# Install the polars package
%pip install polars
%pip install faker

from datetime import *

# Import polars
import polars as pl
from faker import Faker
import random

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/c0/1a/ae019d323e83c6e8a9b4323f3fea94e047715847dfa4c4cbaf20a6f8444e/polars-1.27.1-cp39-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 5.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
class Utility:

    locations = [
        "New York",
        "Los Angeles",
        "Chicago",
        "Houston",
        "Phoenix",
        "Philadelphia",
        "San Antonio",
        "San Diego",
        "Dallas",
        "San Jose",
        "Austin",
        "Jacksonville",
        "Fort Worth",
        "Columbus",
        "Charlotte",
        "San Francisco",
        "Indianapolis",
        "Seattle",
        "Denver",
        "Washington",
        "Boston",
        "El Paso",
        "Nashville",
        "Detroit",
        "Oklahoma City",
        "Portland",
        "Las Vegas",
        "Memphis",
        "Louisville",
        "Baltimore",
        "Milwaukee",
        "Albuquerque",
        "Tucson",
        "Fresno",
        "Sacramento",
        "Kansas City",
        "Mesa",
        "Atlanta",
        "Omaha",
        "Colorado Springs",
        "Raleigh",
        "Miami",
        "Long Beach",
        "Virginia Beach",
        "Oakland",
        "Minneapolis",
        "Tulsa",
        "Arlington",
        "Tampa",
        "New Orleans",
    ]

    @staticmethod
    def generate_sk_values(num: int) -> list[int]:
        """
        Generate a list of SK values.
        """
        return [i for i in range(1, num + 1)]

    @staticmethod
    def generate_names(num: int) -> list[str]:
        """
        Generate a list of first names.
        """
        fake = Faker()
        genders = ["male", "female"]

        full_names = []

        for _ in range(num):

            first_name = last_name = ""
            gender = random.choice(genders)

            if gender == "male":
                first_name = fake.first_name_male()
            else:
                first_name = fake.first_name_female()

            last_name = fake.last_name()
            full_name = f"{first_name} {last_name}"

            full_names.append(full_name)

        return full_names

    @staticmethod
    def generate_emails(full_names: list[str]) -> list[str]:
        emails = []
        """
        Generate a list of ages.
        """

        email_domains = ["gmail.com", "yahoo.com", "outlook.com"]

        for name in full_names:

            email_domain = random.choice(email_domains)

            first_name = name.split()[0].lower()
            last_name = name.split()[-1].lower()

            email = f"{first_name}.{last_name}@{email_domain}"

            emails.append(email)

        return emails

    @staticmethod
    def generate_random_locations(num: int) -> list[str]:
        """
        Generate a list of random locations (with duplicates allowed) from the predefined static variable.
        """
        return [random.choice(Utility.locations) for _ in range(num)]

    @staticmethod
    def generate_entity_ids(prefix: str, num: int) -> list[str]:
        """
        Generate a list of entity-specific IDs with a given prefix.
        
        Args:
            prefix (str): The prefix for the IDs (e.g., "CUST" for customers).
            num (int): The number of IDs to generate.
        
        Returns:
            list[str]: A list of generated IDs.
        """
        return [f"{prefix}{str(i).zfill(5)}" for i in range(1, num + 1)]

    @staticmethod
    def generate_entity_ids(prefix: str, df_length: int) -> list[str]:
        """
        Generate a list of entity-specific IDs with a given prefix.
        
        Args:
            prefix (str): The prefix for the IDs (e.g., "CUST" for customers).
            start (int): The starting number for the IDs.
            end (int): The ending number for the IDs.
        
        Returns:
            list[str]: A list of generated IDs.
        """
        return [
            f"{prefix}_{str(i).zfill(5)}" for i in range(1, df_length + 1)
        ]
    
    @staticmethod
    def generate_random_bools(num: int) -> list[bool]:
        """
        Generate a list of random dates in YYYY-MM-DD format.
        """
        return [random.choice([True, False]) for _ in range(num)]

In [49]:
class Customers(Utility):

    records_count = 100_000

    segments = [
        "Budget Shopper",
        "Premium Shopper",
        "Loyal Customer",
        "First-time Buyer",
        "Occasional Shopper",
        "Deal Seeker",
        "Online Shopper",
        "In-Store Regular",
        "High Value",
        "Churn Risk",
    ]

    @staticmethod
    def generate_random_segments(records_count: int = records_count) -> list[str]:
        """
        Generate a list of random locations (with duplicates allowed) from the predefined static variable.
        """
        return [random.choice(Customers.segments) for _ in range(records_count)]
    
    @classmethod
    def generate_dim_customer_records(cls) -> pl.DataFrame:
        """
        Generate a Polars DataFrame representing the Dim_Customers dimension table.
        
        Returns:
            pl.DataFrame: A dataframe with columns:
                customer_sk, customer_id, first_name, last_name, email, 
                residential_location, customer_segment
        """
        num = cls.records_count
        full_names = cls.generate_names(num)
        emails = cls.generate_emails(full_names)

        # Split full names into first and last names
        first_names = [name.split()[0] for name in full_names]
        last_names = [name.split()[-1] for name in full_names]

        return pl.DataFrame({
            "customer_sk": cls.generate_sk_values(num),
            "customer_id": cls.generate_entity_ids("CUST", num),
            "first_name": first_names,
            "last_name": last_names,
            "email": emails,
            "residential_location": cls.generate_random_locations(num),
            "customer_segment": cls.generate_random_segments(num),
        })

In [50]:
class Products(Utility):

    records_count = 100_000

    product_hierarchy_expanded = {
        "Electronics": {
            "Sony": [
                "Bravia X90J TV",
                "WH-1000XM4 Headphones",
                "PlayStation 5",
                "4K Blu-ray Player",
                "Noise Cancelling Earbuds",
                "Smart Watch",
                "Bluetooth Speaker",
            ],
            "Samsung": [
                "Galaxy S21",
                "Galaxy Tab S8",
                "QLED 4K TV",
                "Smartwatch Active",
                "Galaxy Buds Pro",
                "Portable SSD",
                "Smart Fridge",
            ],
            "Apple": [
                "iPhone 14",
                "MacBook Air M2",
                "iPad Pro",
                "Apple Watch Series 7",
                "AirPods Pro",
                "HomePod Mini",
                "Magic Keyboard",
            ],
            "LG": [
                "OLED TV",
                "LG Gram Laptop",
                "V60 ThinQ Smartphone",
                "UltraGear Monitor",
                "LG Tone Free Headphones",
                "LG Smart Speaker",
                "LG UltraWide Monitor",
            ],
            "Panasonic": [
                "Lumix GH5",
                "Cordless Vacuum",
                "4K Blu-ray Player",
                "Smart Fridge",
                "Portable Speaker",
                "Panasonic Microwave",
                "Soundbar",
            ],
        },
        "Home Appliances": {
            "LG": [
                "Front Load Washing Machine",
                "Smart Refrigerator",
                "Microwave Oven",
                "Dehumidifier",
                "Dishwasher",
                "Air Purifier",
                "Cordless Vacuum Cleaner",
            ],
            "Whirlpool": [
                "Double Door Fridge",
                "Dishwasher 360",
                "Top Load Washer",
                "Air Cooler",
                "Smart Air Fryer",
                "Washing Machine",
                "Induction Cooktop",
            ],
            "Bosch": [
                "Series 4 Dishwasher",
                "Built-in Oven",
                "Cooktop 700",
                "Bosch Refrigerator",
                "Washer Dryer",
                "Smart Coffee Maker",
                "Dishwasher Detergent",
            ],
            "Samsung": [
                "Convertible Refrigerator",
                "Samsung Washing Machine",
                "Dishwasher",
                "Dual Cook Oven",
                "Smart Oven",
                "Smart Air Fryer",
                "Food Processor",
            ],
            "Haier": [
                "Top Load Washing Machine",
                "Air Cooler",
                "Oven Toaster Grill",
                "Deep Freezer",
                "Dehumidifier",
                "Water Purifier",
                "Ice Maker",
            ],
        },
        "Clothing": {
            "Nike": [
                "Air Max 90",
                "Dri-FIT T-Shirt",
                "Running Shorts",
                "Nike Jacket",
                "Air Force 1 Sneakers",
                "Compression Socks",
                "Nike Hoodies",
            ],
            "Adidas": [
                "Ultraboost 22",
                "Climalite Jacket",
                "Training Pants",
                "Superstar Sneakers",
                "Adidas 3-Stripes",
                "Running Tights",
                "Adidas Football Shirt",
            ],
            "Zara": [
                "Slim Fit Shirt",
                "Casual Blazer",
                "Chino Trousers",
                "Belted Coat",
                "Puffer Jacket",
                "Patterned Dress",
                "High-Waist Pants",
            ],
            "Puma": [
                "Clyde Sneakers",
                "Puma Tracksuit",
                "Training Shirt",
                "Running Shoes",
                "Puma Hoodie",
                "Puma Socks",
                "Puma Capri Pants",
            ],
            "Levi's": [
                "501 Original Jeans",
                "Denim Jacket",
                "T-shirt Pack",
                "Chino Shorts",
                "Leather Belt",
                "Bootcut Jeans",
                "Graphic Tee",
            ],
        },
        "Groceries": {
            "Kellogg's": [
                "Corn Flakes",
                "Choco Fills",
                "Muesli 5-in-1",
                "Froot Loops",
                "Rice Krispies",
                "Special K",
                "Granola Bars",
            ],
            "Nestle": [
                "Maggi Noodles",
                "Everyday Milk Powder",
                "KitKat",
                "Nescafé Instant Coffee",
                "Nestle Cornflakes",
                "La Vie Water",
                "Milk Chocolate",
            ],
            "Amul": [
                "Butter 500g",
                "Cheese Slices",
                "Taaza Milk",
                "Ice Cream",
                "Curd",
                "Ghee",
                "Chocolates",
            ],
            "Britannia": [
                "Good Day Biscuits",
                "Milk Bikis",
                "Cheese Slice",
                "Crackers",
                "Bread",
                "Muffins",
                "Little Hearts Biscuits",
            ],
            "Tata": [
                "Tata Tea",
                "Tata Salt",
                "Tata Water",
                "Tata Coffee",
                "Tata Sampann Lentils",
                "Tata Coriander Powder",
                "Tata Sugar",
            ],
        },
        "Furniture": {
            "IKEA": [
                "MALM Bed Frame",
                "BILLY Bookcase",
                "POÄNG Armchair",
                "KALLAX Shelf Unit",
                "LACK Coffee Table",
                "BESTÅ TV Unit",
                "HEMNES Dresser",
            ],
            "Urban Ladder": [
                "Recliner Sofa",
                "Solid Wood Dining Table",
                "Bedside Table",
                "Recliner Chair",
                "Solid Wood Bed",
                "Storage Cabinet",
                "Book Shelf",
            ],
            "Godrej Interio": [
                "Office Desk",
                "Ergonomic Chair",
                "Wardrobe 3-Door",
                "Dining Table Set",
                "Almirah",
                "Sofa Set",
                "Coffee Table",
            ],
            "Home Centre": [
                "Sofa Bed",
                "Coffee Table",
                "Side Table",
                "Teakwood Bed",
                "Wardrobe with Drawers",
                "Recliner",
                "Dining Chair",
            ],
            "Pepperfry": [
                "Sofa Set",
                "Teak Dining Table",
                "Leather Recliner",
                "Storage Cabinet",
                "Book Shelf",
                "Glass Coffee Table",
                "Bar Stools",
            ],
        },
        "Sports & Outdoors": {
            "Decathlon": [
                "Trekking Backpack",
                "Camping Tent",
                "Mountain Bike",
                "Yoga Mat",
                "Tennis Racket",
                "Badminton Shuttlecocks",
                "Football",
            ],
            "Nike": [
                "Football",
                "Running Shoes",
                "Basketball",
                "Yoga Mat",
                "Track Pants",
                "Football Boots",
                "Tennis Shoes",
            ],
            "Adidas": [
                "Soccer Ball",
                "Exercise Dumbbells",
                "Yoga Block",
                "Running Shoes",
                "Tennis Racket",
                "Basketball",
                "Water Bottle",
            ],
            "Puma": [
                "Sports Water Bottle",
                "Football",
                "Running Shoes",
                "Yoga Mat",
                "Tennis Shoes",
                "Track Jacket",
                "Socks",
            ],
            "Reebok": [
                "Dumbbells",
                "Resistance Bands",
                "Tennis Racket",
                "Running Shorts",
                "Training Pants",
                "Yoga Mat",
                "Sweatshirt",
            ],
        },
    }

    @staticmethod
    def generate_dim_products_records(records_count: int = 100_000) -> pl.DataFrame:
        """
        Generate a Polars DataFrame of product records with product_sk, product_id, product_name,
        category, brand, and origin_location.
        """
        products = []
        all_products = []

        # Get all permutations of category, brand, and product_name
        for category, brands in Products.product_hierarchy_expanded.items():
            for brand, items in brands.items():
                for product_name in items:
                    all_products.append(
                        {
                            "category": category,
                            "brand": brand,
                            "product_name": product_name,
                        }
                    )

        # Generate product records
        for i, product in enumerate(all_products, start=1):
            product_sk = i
            product_id = f"PROD_{str(i).zfill(5)}"  # Format like PROD_00001
            origin_location = random.choice(
                Utility.locations
            )  # Generate random location

            # Create the record for this product
            product_record = {
                "product_sk": product_sk,
                "product_id": product_id,
                "product_name": product["product_name"],
                "category": product["category"],
                "brand": product["brand"],
                "origin_location": origin_location,
            }

            products.append(product_record)

            # Stop once the desired records count is reached
            if len(products) >= records_count:
                break

        # Convert the list of product records to a Polars DataFrame
        df = pl.DataFrame(products)

        return df

In [51]:
class SalesPerson(Utility):
    
    records_count = 2000
    
    # Salesperson Types
    salesperson_roles = ["Manager", "Senior Salesperson", "Salesperson", "Sales Associate"]
    
    @staticmethod
    def generate_salesperson_job_roles():
        
        job_roles = []
        
        for i in range (0, 500):
            job_roles.append("Manager")
        
        for i in range(500, 2000):
            job_roles.append(random.choice(SalesPerson.salesperson_roles[1:]))
        
        random.shuffle(job_roles)
        
        return job_roles

    @staticmethod
    def generate_dim_salesperson_records(records_count: int = 2000) -> pl.DataFrame:
        """
        Generate a Polars DataFrame of salesperson records with salesperson_sk, salesperson_id,
        salesperson_name, and salesperson_role.
        """
        salesperson_records = []
        salesperson_names = Utility.generate_names(records_count)  # Generate random salesperson names
        salesperson_roles = SalesPerson.generate_salesperson_job_roles()  # Get salesperson job roles

        # Generate records
        for i in range(records_count):
            salesperson_sk = i + 1  # Sequential SK starting from 1
            salesperson_id = f"SP{str(salesperson_sk).zfill(5)}"  # Format as SP00001, SP00002, etc.
            salesperson_name = salesperson_names[i]
            salesperson_role = salesperson_roles[i]

            # Create the record
            salesperson_record = {
                "salesperson_sk": salesperson_sk,
                "salesperson_id": salesperson_id,
                "salesperson_name": salesperson_name,
                "salesperson_role": salesperson_role
            }

            salesperson_records.append(salesperson_record)

        # Convert the list of salesperson records to a Polars DataFrame
        df = pl.DataFrame(salesperson_records)

        return df

In [52]:
class Stores(Utility):

    store_names = [
        "TechBazaar",
        "HomeHaven",
        "StyleStop",
        "DailyMart",
        "FurniWorld",
        "SportShack",
        "GadgetZone",
        "ApplianceHub",
        "UrbanStyles",
        "FreshNest",
        "FurniCart",
        "GearUp",
        "TrendWay",
        "ElectroPoint",
        "MegaMart",
        "HomeEssence",
        "ClothCorner",
        "ActiveLife",
        "SmartChoice",
        "ValueDepot",
        "QuickCart",
        "FashionFiesta",
        "EcoMart",
        "Roomify",
        "UrbanGear",
        "SmartBuy",
        "Trendify",
        "GrocerEase",
        "FitNGo",
        "TechWorld",
        "FabNest",
        "MegaDeals",
        "ModernMart",
        "BudgetBuy",
        "ZenHome",
        "WearWell",
        "FreshFlow",
        "CosmoMart",
        "FurniSquare",
        "NextGenMart",
        "GoStyle",
        "HyperMart",
        "StyleHub",
        "HomeCentre",
        "DailyDeals",
        "VivaStores",
        "NovaMart",
        "CityMart",
        "ShopEase",
        "NeoStore",
    ]

    records_count = 500

    # Store Types
    store_types = ["Small Stores / Shops", "Supermarkets ", "Large Malls / Complexes"]

    @staticmethod
    def generate_random_store_types(records_count: int = records_count) -> list[str]:
        """
        Generate a list of random store types.
        """
        return [random.choice(Stores.store_types) for _ in range(records_count)]

    @staticmethod
    def generate_random_store_names(records_count: int = records_count) -> list[str]:
        """
        Generate a list of random store names.
        """
        return [random.choice(Stores.store_names) for _ in range(records_count)]

    @staticmethod
    def generate_store_managers_sk(records_count: int = records_count) -> list[str]:
        """
        Generate a list of random store types.
        """
        # Create a list from 1 to 2000
        numbers = list(range(1, 2001))

        # Shuffle the list in-place
        random.shuffle(numbers)

        # Pick the first 500 randomly shuffled numbers
        return numbers[:500]
    
    @staticmethod
    def assign_store_managers(salesperson_df: pl.DataFrame) -> list[int]:
        """
        Assign store managers to the stores.
        """
        # Create a list of store manager SKs
        
        only_managers = salesperson_df.filter(pl.col("sales_role") == "Manager")
        only_manager_sks = only_managers.select("salesperson_sk").to_series().to_list()
        
        return only_manager_sks
    
    @staticmethod
    def generate_dim_store_records(manager_sks: list[int], records_count: int = records_count) -> pl.DataFrame:
        """
        Generate a Polars DataFrame for the Dim_Stores dimension table.

        Args:
            manager_sks (list[int]): List of salesperson SKs with the Manager role.
            records_count (int): Number of store records to generate.

        Returns:
            pl.DataFrame: A Polars DataFrame containing store dimension records.
        """
        if len(manager_sks) < records_count:
            raise ValueError("Number of manager SKs provided is less than the number of stores to generate.")

        # Generate primary keys and entity IDs
        store_sks = list(range(1, records_count + 1))
        store_ids = Stores.generate_entity_ids("STR", records_count)

        # Random values for other fields
        store_names = Stores.generate_random_store_names(records_count)
        store_types = Stores.generate_random_store_types(records_count)
        store_locations = [random.choice(Utility.locations) for _ in range(records_count)]

        # Randomly assign a unique manager SK to each store (sample without replacement)
        store_manager_sks = random.sample(manager_sks, records_count)

        # Create the Polars DataFrame
        return pl.DataFrame({
            "store_sk": store_sks,
            "store_id": store_ids,
            "store_name": store_names,
            "store_type": store_types,
            "store_location": store_locations,
            "store_manager_sk": store_manager_sks,
        })


In [53]:
class Campaigns(Utility):

    records_count = 50

    campaign_names = [
        "Winter Wonders Sale",
        "Spring Splash Discounts",
        "Summer Sizzle Promo",
        "Autumn Deals Drive",
        "Flash Frenzy Friday",
        "Mega Weekend Bonanza",
        "New Year Kickoff",
        "Festive Fever Promo",
        "Back to School Boost",
        "Mid-Year Madness",
        "End of Season Sale",
        "Clearance Countdown",
        "Holiday Shopping Rush",
        "Super Savings Week",
        "Deal of the Month",
        "Loyalty Rewards Blast",
        "Exclusive Insider Offers",
        "Birthday Bash Sale",
        "Weekend Warrior Specials",
        "Price Drop Parade",
        "Black Friday Blowout",
        "Cyber Monday Mania",
        "Green Weekend Deals",
        "Red Tag Clearance",
        "Customer Appreciation Week",
        "VIP Access Sale",
        "Limited Time Treasures",
        "Golden Hour Bargains",
        "Final Call Frenzy",
        "Instant Gratification Sale",
        "Double Points Bonanza",
        "Cashback Carnival",
        "New Product Launch Promo",
        "Spring Refresh Event",
        "Winter Clearance Carnival",
        "Shop & Save Spectacular",
        "Early Bird Specials",
        "Mystery Discount Days",
        "Midnight Madness",
        "12 Days of Deals",
        "Countdown to Savings",
        "New Arrival Showcase",
        "Hot Picks Promo",
        "Ultimate Value Event",
        "Happy Hour Discounts",
        "Seasonal Surprise Box",
        "Gift More, Spend Less",
        "Cart Crasher Campaign",
        "BOGO Blitz",
        "Smart Shopper Week",
    ]
    
    @staticmethod
    def generate_campaigns_with_dates(campaign_names, dates_list):
        campaigns = []
        for name in campaign_names:
            start_date = random.choice(dates_list)
            # Add 7 to 30 days, ensuring it doesn’t go beyond the date list's range
            possible_end_dates = [d for d in dates_list if d > start_date]
            if not possible_end_dates:
                continue  # Skip if no valid end date
            end_date = random.choice(possible_end_dates)
            campaigns.append({
                "campaign_name": name,
                "start_date": start_date,
                "end_date": end_date
            })
        return campaigns

    @staticmethod
    def generate_dim_campaign_records(dates_df: pl.DataFrame) -> pl.DataFrame:
        dates_list = dates_df["full_date"].to_list()
        campaigns_raw = Campaigns.generate_campaigns_with_dates(Campaigns.campaign_names, dates_list)

        campaigns_df = pl.DataFrame(campaigns_raw).with_columns([
            pl.arange(1, len(campaigns_raw)+1).alias("campaign_sk"),
            pl.concat_str([pl.lit("CAMP_"), pl.arange(1, len(campaigns_raw)+1).cast(pl.Utf8).str.zfill(3)]).alias("campaign_id"),
            pl.lit(None).alias("campaign_budget")  # Placeholder
        ])

        # Join to get start_date_sk and end_date_sk
        date_sk_df = dates_df.select(["full_date", "date_sk"])
        campaigns_df = campaigns_df.join(
            date_sk_df.rename({"full_date": "start_date", "date_sk": "start_date_sk"}),
            on="start_date"
        )
        campaigns_df = campaigns_df.join(
            date_sk_df.rename({"full_date": "end_date", "date_sk": "end_date_sk"}),
            on="end_date"
        )

        # Add campaign_budget now
        # campaigns_df = campaigns_df.with_columns([
        #     pl.int_range(10000, 10000 + len(campaigns_raw)).shuffle().alias("campaign_budget")
        # ])
        # campaigns_df = campaigns_df.with_columns([
        #     pl.int_range(100_000, 1_000_000).sample_n(len(campaigns_raw), with_replacement=False).alias("campaign_budget")
        # ])
        
        # Generate budgets externally
        campaign_budgets = random.sample(range(100_000, 1_000_000), len(campaigns_raw))

        # Add to the DataFrame
        campaigns_df = campaigns_df.with_columns([
            pl.Series("campaign_budget", campaign_budgets)
        ])


        # Final column ordering
        campaigns_df = campaigns_df.select([
            "campaign_sk", "campaign_id", "campaign_name", "start_date", "end_date",
            "start_date_sk", "end_date_sk", "campaign_budget"
        ])

        return campaigns_df

In [54]:
class Dates:

    @staticmethod
    def generate_dates_start_to_end(
        start_date: str,
        end_date: str,
        randomize_time: bool = False,
        list_length: int = None
    ) -> list[str]:
        """
        Generate a list of ISO8601 datetime strings (YYYY-MM-DDTHH:MM:SS)
        between start_date and end_date.

        If list_length is specified, it generates that many random dates (with possible duplicates).
        If randomize_time is True, it adds random HH:MM:SS.
        """
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        delta_days = (end - start).days + 1

        all_possible_dates = [start + timedelta(days=i) for i in range(delta_days)]

        if list_length is None:
            selected_dates = all_possible_dates
        else:
            selected_dates = [
                random.choice(all_possible_dates) for _ in range(list_length)
            ]

        result_dates = []
        for date in selected_dates:
            if randomize_time:
                random_time = timedelta(
                    hours=random.randint(0, 23),
                    minutes=random.randint(0, 59),
                    seconds=random.randint(0, 59),
                )
                date += random_time

            result_dates.append(date.isoformat(timespec="seconds"))

        return result_dates
    
    @staticmethod
    def generate_date_df(start_date: str, end_date: str) -> pl.DataFrame:
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
        end = datetime.strptime(end_date, "%Y-%m-%d").date()

        date_series = pl.date_range(start=start, end=end, interval="1d", eager=True)

        df = pl.DataFrame({
            "full_date": date_series
        }).with_columns([
            pl.arange(1, len(date_series) + 1).alias("date_sk"),
            pl.col("full_date").dt.year().alias("year"),
            pl.col("full_date").dt.month().alias("month"),
            pl.col("full_date").dt.day().alias("day"),
            pl.col("full_date").dt.weekday().alias("weekday"),
            pl.col("full_date").dt.quarter().alias("quarter"),
        ])

        return df


# Optional: keep this for semantic clarity if needed in your project
class DateDimensionGenerator:

    @staticmethod
    def generate_date_df(start_date: str, end_date: str) -> pl.DataFrame:
        return Dates.generate_date_df(start_date, end_date)


In [55]:
customers_df = Customers.generate_dim_customer_records()

In [56]:
customers_df

customer_sk,customer_id,first_name,last_name,email,residential_location,customer_segment
i64,str,str,str,str,str,str
1,"""CUST_00001""","""Chelsey""","""Francis""","""chelsey.francis@gmail.com""","""Boston""","""High Value"""
2,"""CUST_00002""","""Chad""","""Herrera""","""chad.herrera@gmail.com""","""San Antonio""","""Churn Risk"""
3,"""CUST_00003""","""Calvin""","""Hale""","""calvin.hale@yahoo.com""","""Fort Worth""","""Churn Risk"""
4,"""CUST_00004""","""Richard""","""Smith""","""richard.smith@outlook.com""","""Denver""","""First-time Buyer"""
5,"""CUST_00005""","""Daniel""","""Mays""","""daniel.mays@yahoo.com""","""Virginia Beach""","""Online Shopper"""
…,…,…,…,…,…,…
99996,"""CUST_99996""","""Melissa""","""Burnett""","""melissa.burnett@outlook.com""","""Albuquerque""","""High Value"""
99997,"""CUST_99997""","""Teresa""","""Williams""","""teresa.williams@outlook.com""","""Omaha""","""Loyal Customer"""
99998,"""CUST_99998""","""Tony""","""Robertson""","""tony.robertson@outlook.com""","""Columbus""","""Deal Seeker"""


In [57]:
products_df = Products.generate_dim_products_records()

In [58]:
products_df

product_sk,product_id,product_name,category,brand,origin_location
i64,str,str,str,str,str
1,"""PROD_00001""","""Bravia X90J TV""","""Electronics""","""Sony""","""Kansas City"""
2,"""PROD_00002""","""WH-1000XM4 Headphones""","""Electronics""","""Sony""","""San Francisco"""
3,"""PROD_00003""","""PlayStation 5""","""Electronics""","""Sony""","""San Jose"""
4,"""PROD_00004""","""4K Blu-ray Player""","""Electronics""","""Sony""","""Los Angeles"""
5,"""PROD_00005""","""Noise Cancelling Earbuds""","""Electronics""","""Sony""","""Memphis"""
…,…,…,…,…,…
206,"""PROD_00206""","""Tennis Racket""","""Sports & Outdoors""","""Reebok""","""Kansas City"""
207,"""PROD_00207""","""Running Shorts""","""Sports & Outdoors""","""Reebok""","""Las Vegas"""
208,"""PROD_00208""","""Training Pants""","""Sports & Outdoors""","""Reebok""","""San Jose"""


In [59]:
salespersons_df = SalesPerson.generate_dim_salesperson_records()

In [60]:
salespersons_df

salesperson_sk,salesperson_id,salesperson_name,salesperson_role
i64,str,str,str
1,"""SP00001""","""John Norris""","""Salesperson"""
2,"""SP00002""","""Richard Avery""","""Manager"""
3,"""SP00003""","""Julie Avila""","""Manager"""
4,"""SP00004""","""Rhonda Mccarthy""","""Salesperson"""
5,"""SP00005""","""Kathryn Jackson""","""Salesperson"""
…,…,…,…
1996,"""SP01996""","""Carla Ramsey""","""Sales Associate"""
1997,"""SP01997""","""Kimberly Brown""","""Senior Salesperson"""
1998,"""SP01998""","""Elizabeth Greene""","""Salesperson"""


In [61]:
manager_sks = salespersons_df.filter(pl.col("salesperson_role") == "Manager").select("salesperson_sk").to_series().to_list()

In [62]:
stores_df = Stores.generate_dim_store_records(manager_sks)

In [63]:
stores_df

store_sk,store_id,store_name,store_type,store_location,store_manager_sk
i64,str,str,str,str,i64
1,"""STR_00001""","""TechWorld""","""Large Malls / Complexes""","""Los Angeles""",670
2,"""STR_00002""","""Roomify""","""Small Stores / Shops""","""Baltimore""",171
3,"""STR_00003""","""EcoMart""","""Supermarkets ""","""Phoenix""",12
4,"""STR_00004""","""TechBazaar""","""Small Stores / Shops""","""Tampa""",1450
5,"""STR_00005""","""FurniCart""","""Large Malls / Complexes""","""San Antonio""",175
…,…,…,…,…,…
496,"""STR_00496""","""GrocerEase""","""Small Stores / Shops""","""Long Beach""",291
497,"""STR_00497""","""ShopEase""","""Large Malls / Complexes""","""Virginia Beach""",659
498,"""STR_00498""","""GoStyle""","""Small Stores / Shops""","""Washington""",512


In [64]:
dates_df = Dates.generate_date_df("2024-01-01", "2024-12-31")


In [65]:
dates_df

full_date,date_sk,year,month,day,weekday,quarter
date,i64,i32,i8,i8,i8,i8
2024-01-01,1,2024,1,1,1,1
2024-01-02,2,2024,1,2,2,1
2024-01-03,3,2024,1,3,3,1
2024-01-04,4,2024,1,4,4,1
2024-01-05,5,2024,1,5,5,1
…,…,…,…,…,…,…
2024-12-27,362,2024,12,27,5,4
2024-12-28,363,2024,12,28,6,4
2024-12-29,364,2024,12,29,7,4


In [74]:
campaigns_df = Campaigns.generate_dim_campaign_records(dates_df)

In [76]:
campaigns_df = campaigns_df.drop(["start_date", "end_date"], )

In [77]:
campaigns_df

campaign_sk,campaign_id,campaign_name,start_date_sk,end_date_sk,campaign_budget
i64,str,str,i64,i64,i64
6,"""CAMP_006""","""Mega Weekend Bonanza""",37,41,231714
24,"""CAMP_024""","""Red Tag Clearance""",27,55,701892
39,"""CAMP_039""","""Midnight Madness""",30,91,396601
36,"""CAMP_036""","""Shop & Save Spectacular""",15,101,287542
47,"""CAMP_047""","""Gift More, Spend Less""",104,141,585808
…,…,…,…,…,…
35,"""CAMP_035""","""Winter Clearance Carnival""",203,352,640535
3,"""CAMP_003""","""Summer Sizzle Promo""",201,355,480660
2,"""CAMP_002""","""Spring Splash Discounts""",98,356,372105


In [78]:
class Fact_Sales:
    records_count = 1_000_000
    
    @staticmethod
    def generate_fact_sales_records(
        customers_df: pl.DataFrame,
        products_df: pl.DataFrame,
        stores_df: pl.DataFrame,
        salesperson_df: pl.DataFrame,
        campaigns_df: pl.DataFrame,
        dates_df: pl.DataFrame,
        is_denormalized: bool = True
    ) -> pl.DataFrame:
        """
        Generate a Polars DataFrame for the Fact_Sales table.
        If is_denormalized is True, include denormalized fields from dimension tables.
        """
        # Generate random SKs for each dimension
        customer_sks = random.choices(customers_df["customer_sk"].to_list(), k=Fact_Sales.records_count)
        product_sks = random.choices(products_df["product_sk"].to_list(), k=Fact_Sales.records_count)
        store_sks = random.choices(stores_df["store_sk"].to_list(), k=Fact_Sales.records_count)
        salesperson_sks = random.choices(salesperson_df["salesperson_sk"].to_list(), k=Fact_Sales.records_count)
        campaign_sks = random.choices(campaigns_df["campaign_sk"].to_list(), k=Fact_Sales.records_count)

        # Map campaign_sk to start_date_sk and end_date_sk
        campaign_date_map = campaigns_df.select(["campaign_sk", "start_date_sk", "end_date_sk"]).to_dict(as_series=False)

        # Generate sales_date based on campaign dates
        sales_dates = []
        for campaign_sk in campaign_sks:
            start_date_sk = campaign_date_map["start_date_sk"][campaign_sk - 1]
            end_date_sk = campaign_date_map["end_date_sk"][campaign_sk - 1]

            start_date = dates_df.filter(pl.col("date_sk") == start_date_sk)["full_date"].to_list()[0]
            end_date = dates_df.filter(pl.col("date_sk") == end_date_sk)["full_date"].to_list()[0]

            start_date_obj = datetime.fromisoformat(str(start_date))
            end_date_obj = datetime.fromisoformat(str(end_date))
            random_date = start_date_obj + timedelta(days=random.randint(0, (end_date_obj - start_date_obj).days))

            random_time = timedelta(
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59)
            )
            sales_dates.append((random_date + random_time).isoformat())

        total_amounts = [round(random.uniform(500, 5000), 2) for _ in range(Fact_Sales.records_count)]
        sales_sks = list(range(1, Fact_Sales.records_count + 1))
        sales_ids = [f"SALES_{str(i).zfill(7)}" for i in sales_sks]

        # Base columns
        fact_sales_dict = {
            "sales_sk": sales_sks,
            "sales_id": sales_ids,
            "customer_sk": customer_sks,
            "product_sk": product_sks,
            "store_sk": store_sks,
            "salesperson_sk": salesperson_sks,
            "campaign_sk": campaign_sks,
            "sales_date": sales_dates,
            "total_amount": total_amounts
        }

        fact_sales_df = pl.DataFrame(fact_sales_dict)

        if is_denormalized:
            # Convert to DataFrame for efficient join
            fact_sales_df = (
                fact_sales_df
                .join(customers_df.select(["customer_sk", "customer_segment"]), on="customer_sk", how="left")
                .join(products_df.select(["product_sk", "category"]), on="product_sk", how="left")
                .join(stores_df.select(["store_sk", "store_type", "store_location"]), on="store_sk", how="left")
                .join(campaigns_df.select(["campaign_sk", "campaign_name"]), on="campaign_sk", how="left")
            )

        return fact_sales_df


In [80]:
fact_sales_denormalized_df = Fact_Sales.generate_fact_sales_records(
    customers_df=customers_df,
    products_df=products_df,
    stores_df=stores_df,
    salesperson_df=salespersons_df,
    campaigns_df=campaigns_df,
    dates_df=dates_df,
    is_denormalized=True
)

In [167]:
fact_sales_denormalized_df

sales_sk,sales_id,customer_sk,product_sk,store_sk,salesperson_sk,campaign_sk,sales_date,total_amount,customer_segment,category,store_type,store_location,campaign_name
i64,str,i64,i64,i64,i64,i64,str,f64,str,str,str,str,str
1,"""SALES_0000001""",49008,182,256,32,15,"""2024-06-05T07:07:10""",1875.46,"""Budget Shopper""","""Sports & Outdoors""","""Large Malls / Complexes""","""Tampa""","""Deal of the Month"""
2,"""SALES_0000002""",51434,51,484,249,39,"""2024-11-24T10:16:58""",2017.15,"""Budget Shopper""","""Home Appliances""","""Supermarkets ""","""Arlington""","""Midnight Madness"""
3,"""SALES_0000003""",75043,172,297,1821,11,"""2024-03-17T15:29:23""",4075.78,"""High Value""","""Furniture""","""Small Stores / Shops""","""Atlanta""","""End of Season Sale"""
4,"""SALES_0000004""",88794,46,101,231,38,"""2024-12-13T16:04:45""",3355.69,"""Occasional Shopper""","""Home Appliances""","""Small Stores / Shops""","""Philadelphia""","""Mystery Discount Days"""
5,"""SALES_0000005""",22107,162,359,834,38,"""2024-11-28T05:01:17""",3855.56,"""First-time Buyer""","""Furniture""","""Small Stores / Shops""","""Tucson""","""Mystery Discount Days"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
999996,"""SALES_0999996""",12257,140,17,1325,15,"""2024-07-08T20:59:59""",520.77,"""Occasional Shopper""","""Groceries""","""Large Malls / Complexes""","""Denver""","""Deal of the Month"""
999997,"""SALES_0999997""",91932,208,265,1714,13,"""2024-08-19T10:50:02""",4154.09,"""Premium Shopper""","""Sports & Outdoors""","""Supermarkets ""","""Detroit""","""Holiday Shopping Rush"""
999998,"""SALES_0999998""",64138,160,75,440,44,"""2024-12-15T13:46:43""",2573.77,"""Loyal Customer""","""Furniture""","""Supermarkets ""","""Minneapolis""","""Ultimate Value Event"""


In [81]:
# Drop specified columns from the fact_sales_denormalized_df DataFrame
fact_sales_normalized_df = fact_sales_denormalized_df.drop(
    ["customer_segment", "category", "store_type", "store_location", "campaign_name"]
)

In [82]:
fact_sales_normalized_df

sales_sk,sales_id,customer_sk,product_sk,store_sk,salesperson_sk,campaign_sk,sales_date,total_amount
i64,str,i64,i64,i64,i64,i64,str,f64
1,"""SALES_0000001""",56504,167,191,1442,3,"""2024-02-19T15:28:02""",2421.54
2,"""SALES_0000002""",59945,100,422,1996,8,"""2024-03-14T09:07:51""",2487.22
3,"""SALES_0000003""",54709,71,411,1388,35,"""2024-10-02T13:50:38""",2915.91
4,"""SALES_0000004""",77739,137,242,1211,6,"""2024-05-06T20:17:06""",4086.9
5,"""SALES_0000005""",97840,207,380,570,19,"""2024-08-23T22:09:16""",2425.33
…,…,…,…,…,…,…,…,…
999996,"""SALES_0999996""",40375,111,355,61,6,"""2024-04-16T13:23:43""",912.08
999997,"""SALES_0999997""",79131,69,442,580,43,"""2024-08-31T01:10:12""",2636.94
999998,"""SALES_0999998""",28642,133,211,1559,11,"""2024-05-27T21:43:05""",4360.29


In [83]:
# Export all DataFrames to CSV files
customers_df.write_csv("./data/with_header/dim_customers.csv")
products_df.write_csv("./data/with_header/dim_products.csv")
salespersons_df.write_csv("./data/with_header/dim_salesperson.csv")
stores_df.write_csv("./data/with_header/dim_stores.csv")
dates_df.write_csv("./data/with_header/dim_dates.csv")
campaigns_df.write_csv("./data/with_header/dim_campaigns.csv")
fact_sales_normalized_df.write_csv("./data/with_header/fact_sales_normalized.csv")
fact_sales_denormalized_df.write_csv("./data/with_header/fact_sales_denormalized.csv")

In [84]:
# Export all DataFrames to CSV files
customers_df.write_csv("./data/without_header/dim_customers.csv")
products_df.write_csv("./data/without_header/dim_products.csv")
salespersons_df.write_csv("./data/without_header/dim_salesperson.csv")
stores_df.write_csv("./data/without_header/dim_stores.csv")
dates_df.write_csv("./data/without_header/dim_dates.csv")
campaigns_df.write_csv("./data/without_header/dim_campaigns.csv")
fact_sales_normalized_df.write_csv("./data/without_header/fact_sales_normalized.csv")
fact_sales_denormalized_df.write_csv("./data/without_header/fact_sales_denormalized.csv")